In [1]:
import pandas as pd
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from plotly.subplots import make_subplots
# from pyngrok import ngrok, conf

In [2]:
merged_df = pd.read_csv("Processed.csv", encoding="utf-8")

merged_df = merged_df[merged_df['StatisticGroup'] != 'שגיאת הזנה']
merged_df = merged_df[merged_df['GeoName'] != "מקום אחר"]
merged_df = merged_df[merged_df["PoliceDistrict"] != 'כל הארץ']
merged_df = merged_df[merged_df["StatisticGroup"] != 'שאר עבירות']
merged_df = merged_df[merged_df["StatisticGroup"] != 'סעיפי הגדרה']


# merged_df['QuarterNumber'] = merged_df['Quarter'].str.extract('Q([1-4])').astype(int)
# merged_df = merged_df.sort_values(by=['Year', 'QuarterNumber'])


In [3]:
# Dashboard 1 Preprocessing: Crime Trends by Police District
war_effects_df = merged_df.copy()
war_effects_df = war_effects_df[war_effects_df['StatisticGroup'] != "שאר עבירות"]
war_effects_df = war_effects_df[war_effects_df['StatisticGroup'] != "סעיפי הגדרה"]


statis_group_categories = war_effects_df['StatisticGroup'].unique()

war_effects_first_drop_down = merged_df['PoliceDistrict'].unique()
war_effects_second_drop_down = merged_df[merged_df['PoliceDistrict'] == war_effects_first_drop_down[0]]['PoliceMerhav'].unique()
war_effects_third_drop_down = merged_df[(merged_df['PoliceDistrict'] == war_effects_first_drop_down[0]) & (merged_df['PoliceMerhav'] == war_effects_second_drop_down[0])]['Yeshuv'].unique()


In [4]:
# Dashboard 2 Preprocessing: Crime cases

# Filter pre- and post-war DataFrames
pre_war_df = merged_df[
    ((merged_df["Year"] == 2022) & (merged_df["Quarter"] == "Q4")) |
    ((merged_df["Year"] == 2023) & (merged_df["Quarter"].isin(["Q1", "Q2", "Q3"])))
]
post_war_df = merged_df[
    ((merged_df["Year"] == 2023) & (merged_df["Quarter"] == "Q4")) |
    ((merged_df["Year"] == 2024) & (merged_df["Quarter"].isin(["Q1", "Q2", "Q3"])))
]

# Group by StatisticGroup and count rows
pre_war_counts = pre_war_df.groupby('StatisticGroup').size().reset_index(name='Count')
post_war_counts = post_war_df.groupby('StatisticGroup').size().reset_index(name='Count')

pre_war_counts = pre_war_counts.sort_values('StatisticGroup')
post_war_counts = post_war_counts.sort_values('StatisticGroup')

In [5]:
# Dashboard 3 Preprocessing: Security Cases
bitahon_df = merged_df[merged_df["StatisticGroup"] == "עבירות בטחון"]
bitahon_df['YearQuarter'] = (bitahon_df['Quarter'].astype(str) + '/' + bitahon_df['Year'].astype(str))

# Aggregated data for before and after a specific time period
bitahon_df['TimePeriod'] = bitahon_df['YearQuarter'].apply(lambda x: 'לפני' if x in ["Q4/2022", "Q1/2023", "Q2/2023", "Q3/2023"] else 'אחרי')
security_statistic_type = bitahon_df["StatisticType"].unique()

# Group data by StatisticType and TimePeriod
twin_bar_data = bitahon_df.groupby(['StatisticType', 'TimePeriod']).size().reset_index(name='CrimeAmount')


# Calculate the total sum of CrimeAmount for each StatisticType
total_sums = twin_bar_data.groupby('StatisticType')['CrimeAmount'].sum().reset_index(name='TotalCrimeAmount')

# Sort twin_bar_data by the total sum of CrimeAmount
twin_bar_data = twin_bar_data.merge(total_sums, on='StatisticType')  # Add the total sum as a column
twin_bar_data = twin_bar_data.sort_values(by='TotalCrimeAmount', ascending=True)

geo_names = sorted(bitahon_df["GeoName"].unique(), key=str)

def make_security_cases_fig(df, is_log=False, highlight_statistic=None):
    # If log transformation is needed, apply log to the 'CrimeAmount' column
    if is_log:
        df = df.copy()
        df['LogCrimeAmount'] = df['CrimeAmount'].apply(lambda x: max(1, x)).apply(np.log)

    # Filter data for highlighted and dimmed bars
    highlighted_data = df[df['StatisticType'] == highlight_statistic] if highlight_statistic else pd.DataFrame()
    dimmed_data = df[df['StatisticType'] != highlight_statistic] if highlight_statistic else df

    # Create the dimmed trace
    fig = px.bar(
        dimmed_data,
        x='StatisticType',
        y='CrimeAmount' if not is_log else 'LogCrimeAmount',
        color='TimePeriod',
        barmode='group',
        opacity=0.4,  # Reduced opacity for dimmed bars
        category_orders={'StatisticType': df['StatisticType'].unique(), 'TimePeriod': ["לפני", "אחרי"]},
        labels={'StatisticType': 'סוג פשיעה', 'CrimeAmount': "מס' תיקי פשיעה (log scale)" if is_log else "מס' תיקי פשיעה"},
        # height=400,
        # width=1000,
        title="עבירות ביטחון לפני ואחרי המלחמה",
        custom_data=["TimePeriod", "CrimeAmount"]  # Ensure hover data is passed correctly
    )

    # Add the highlighted trace (if applicable)
    if not highlighted_data.empty:
        highlighted_trace = px.bar(
            highlighted_data,
            x='StatisticType',
            y='CrimeAmount' if not is_log else 'LogCrimeAmount',
            color='TimePeriod',
            barmode='group',
            opacity=1.0,  # Full opacity for highlighted bars
            category_orders={'StatisticType': df['StatisticType'].unique(), 'TimePeriod': ["לפני", "אחרי"]},
            labels={'StatisticType': 'סוג פשיעה', 'CrimeAmount': "מס' תיקי פשיעה (log scale)" if is_log else "מס' תיקי פשיעה"},
            custom_data=["TimePeriod", "CrimeAmount"]  # Ensure hover data is passed correctly

        )

        # Add highlighted trace to the figure
        for trace in highlighted_trace.data:
            trace.showlegend = False
            fig.add_trace(trace)
            

    # Update hover template for all traces
    for trace in fig.data:
        # trace.hovertemplate = (
        #     "זמן: %{customdata[0]}<br>"
        #     "סוג פשיעה: %{x}<br>"
        #     "מס' תיקי פשיעה: %{customdata[1]}<extra></extra>"
        # )
        trace.hovertemplate =("מס' תיקי פשיעה %{customdata[0]} המלחמה: %{customdata[1]}<extra></extra>")

    fig.update_yaxes(side="right", title = None)

    # Adjust layout settings
    fig.update_layout(
        legend=dict(
            x=-0.05,  # Center the legend horizontally
            y=0.5,
            xanchor="center",  # Anchor the x position to the center
            yanchor="middle"  # Anchor the y position to the top
        ),
        legend_title=dict(
            text="זמן",  # The new legend title text
            font=dict(size=18, family="Arial"),  # Customize the font of the title
            side="top right"
        ),
        title={
            'y': 0.85,  # Vertical alignment of the title (closer to 1 moves it up, closer to 0 moves it down)
            'x': 0.5,  # Horizontal alignment of the title (0.5 centers it)
            'xanchor': 'center',
            'yanchor': 'top',
        },
            # autosize=False,  # Ensure the graph size doesn't change dynamically
            height=400,  # Fix the height
            # width = 1150
            # width=None # Use full width of the container, or set an explicit value like 800
            # responsive=False  # Disable responsiveness
            width = 1000
    )

    # Add annotation for Y-axis title
    fig.add_annotation(
        x=1.08,  # Align to the right
        y=0.5,  # Center vertically
        text="מס' תיקי פשיעה (לוגריתמי)" if is_log else "מס' תיקי פשיעה",  # Adjust the label based on scale
        showarrow=False,
        textangle=-270,  # Rotate vertically
        xref="paper",  # Relative to the figure
        yref="paper",
        font=dict(size=15, family="Arial")
    )

    return fig


C:\Users\moanz\AppData\Local\Temp\ipykernel_2636\72650883.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bitahon_df['YearQuarter'] = (bitahon_df['Quarter'].astype(str) + '/' + bitahon_df['Year'].astype(str))
C:\Users\moanz\AppData\Local\Temp\ipykernel_2636\72650883.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bitahon_df['TimePeriod'] = bitahon_df['YearQuarter'].apply(lambda x: 'לפני' if x in ["Q4/2022", "Q1/2023", "Q2/2023", "Q3/2023"] else 'אחרי')


In [6]:
# Dashboard 4 Preprocessing: heat of war


# Load the dataset
file_path = 'war23_alarms_monthly.csv'
data = pd.read_csv(file_path)

from_loc_to_geonome_mapping = {
    'חיפה - נווה שאנן ורמות כרמל':'חיפה',
    'חיפה - כרמל': 'חיפה',
    'חיפה - קריית חיים ושמואל': 'חיפה',
    'חיפה - מפרץ': 'חיפה',
    'חיפה - כרמל ועיר תחתית': 'חיפה',
    'חיפה - מערב': 'חיפה',
    'קריית שמונה': 'קרית שמונה',
    'תל אביב - דרום העיר ויפו': 'תל אביב יפו',
    'תל אביב - מרכז העיר': 'תל אביב יפו',
    'תל אביב - עבר הירקון': 'תל אביב יפו',
    'תל אביב - מזרח': 'תל אביב יפו',

    'נתניה - מערב': 'נתניה',
    'נתניה - מזרח': 'נתניה',

    'אזור תעשייה חבל מודיעין': 'מודיעין-מכבים-רעות',
    'מודיעין - ישפרו סנטר': 'מודיעין-מכבים-רעות',
    'מודיעין': 'מודיעין-מכבים-רעות',
    'מודיעין מכבים רעות': 'מודיעין-מכבים-רעות',
    'מודיעין - ליגד סנטר': 'מודיעין-מכבים-רעות',

    'ירושלים - אזור תעשייה עטרות': 'ירושלים',
    'ירושלים - מזרח': 'ירושלים',
    'ירושלים - צפון': 'ירושלים',
    'ירושלים - כפר עקב': 'ירושלים',
    'ירושלים - מרכז': 'ירושלים',
    'ירושלים - מערב': 'ירושלים',
    'ירושלים - דרום': 'ירושלים',

    'אזור תעשייה הדרומי אשקלון': 'אשקלון',
    'אשקלון - דרום': 'אשקלון',
    'אזור תעשייה צפוני אשקלון': 'אשקלון',
    'אשקלון - צפון': 'אשקלון',

    'באר שבע - מזרח': 'באר שבע',
    'באר שבע - דרום': 'באר שבע',
    'באר שבע - צפון': 'באר שבע',
    'באר שבע - מערב': 'באר שבע',
}

data['loc'] = data['loc'].replace(from_loc_to_geonome_mapping)

aggregated_df = data.groupby('loc', as_index=False).sum()

# If you want to replace the old rows with the new summed rows
# This will keep the rows where 'mapped_col' is unique and summed
data = aggregated_df



# Extract columns with monthly data
monthly_columns = [col for col in data.columns if '-' in col]

# Define quarters and map months to their respective quarters
quarters = {
    'Q1': ['01', '02', '03'],
    'Q2': ['04', '05', '06'],
    'Q3': ['07', '08', '09'],
    'Q4': ['10', '11', '12']
}

# Extract available years from the monthly columns
years = sorted(set(col.split('-')[0] for col in monthly_columns))

# Process each year to add valid quarters
for year in years:
    for quarter, months in quarters.items():
        # Generate month column names for the current year and quarter
        quarter_columns = [f"{year}-{month}" for month in months]
        # Check if all required months for the quarter are present in the data
        if all(col in monthly_columns for col in quarter_columns):
            # Create the quarterly column by summing the corresponding months
            data[f"{year}_{quarter}"] = data[quarter_columns].sum(axis=1)

# Display the updated dataset
set_a = set(data["loc"])
set_b = set(post_war_df["GeoName"])

# Find overlap
overlap = set_a.intersection(set_b)



count_df = post_war_df.groupby(['GeoName', 'Year', 'Quarter']).size().reset_index(name='Count')


points = []
quarters_to_include_data = ['2023_Q4', '2024_Q1', '2024_Q2', '2024_Q3']
count_filters_year_q = [(2023, 'Q4'), (2024, 'Q1'), (2024, 'Q2'), (2024, 'Q3')]


for city in overlap:
    for i in range(4):
        filtered_dataf = data[
            (data['loc'] == city)
        ][quarters_to_include_data[i]]
        x = filtered_dataf.iloc[0]

        filtered_df = count_df[
            (count_df['GeoName'] == city) &
            (count_df['Year'] == count_filters_year_q[i][0]) &
            (count_df['Quarter'] == count_filters_year_q[i][1])
            ]

        y = filtered_df['Count'].iloc[0]

        points.append((city, count_filters_year_q[i][0], count_filters_year_q[i][1], x, y))
# Convert points list to DataFrame for easier plotting


df = pd.DataFrame(points, columns=['City', 'Year', 'Quarter', 'x', 'y'])




In [7]:
def overview_layout(): 
    layout = html.Table( 
        style={
            "width": "100%",  # Full page width
            "height": "100vh",  # Full viewport height
            "display": "flex",  # Use flexbox for centering
            "justify-content": "center",  # Center horizontally
            # "align-items": "center"  # Center vertically
        },
        children=[ 
            html.Tr(
                html.Td( 
                    style={
                        "text-align": "right",  # Center text horizontally inside the cell
                        "vertical-align": "middle",  # Center text vertically inside the cell
                        "width": "80%",


                    }, 
                    className="dash-tabs-content", 
                    children=dcc.Markdown(
                        '''
                        # 📊 פשיעה בצל מלחמה 
                        ### מגישים: אליה אהרון, נועם בן משה, רומי ריכטר, תומר רונן

                        ## 📝 מבוא  
                        פרויקט זה עוסק בניתוח נתוני פשיעה בישראל בשנים 2022-2024, תוך בחינת השפעת מלחמת 'חרבות ברזל'.  
                        מטרת הפרויקט: בחינת השפעת מלחמת חרבות ברזל על דפוסי הפשיעה במדינת ישראל.

                        🔍 **עיקרי הפרויקט**:  
                        - בחינת שינויי פשיעה לפני ואחרי המלחמה.  
                        - זיהוי אזורים וסוגי עבירות שדורשים מיקוד משאבים.  
                        - עיצוב ויזואליזציות להנגשת תובנות בצורה ברורה.  

                        ## 📂 נתונים  
                        1. **תיקי פשיעה בישראל**  
                        הנתונים לפרויקט נלקחו מאתר data.gov.il. בחרנו להשתמש בנתוני תיקי הפשיעה מהשנים 2022-2024, וסיננו את הרשומות כך שיכילו 4 רבעונים לפני פרוץ המלחמה ו4 רבעונים לאחר פרוץ המלחמה. 
                        כל רשומה מייצגת תיק פשיעה עם תיאור של העבירה, המיקום הגיאוגרפי והרבעון שבוצעה.
                        
                        2. **עצימות מלחמה**  
                        נתונים על כמות אזעקות ('צבע אדום') בכל יישוב מאוקטובר 2023 ועד ינואר 2025. לצורך הויזואליזציה הרביעית.

                        ## ✅ מטלות הפרויקט  
                        1. **סקירת סוגי תיקי הפשיעה בישראל**  
                        🔍 השוואה של כמות העבירות לפני ואחרי פרוץ המלחמה, על פני 2 רמות של סוגי העבירות הקיימות בנתונים.
                        2. **מגמות הפשיעה בחלוקה גיאוגרפית**  
                        📈 בנתונים 3 רמות המתארות תחומי אחריות משטרתיים. הויזואליזציה תאפשר להציג למשתמש את המגמות של כל סוגי תיקי הפשיעה במשך שמונת הרבועונים, ולאפשר השוואה של מגמות הפשיעה בין רמות אחריות דומות.
                        3. **השפעת המלחמה על עבירות ביטחון **  
                        🛡 המטרה היא להמחיש את השינוי שחל בתיקי עבירות הביטחון בהשפעת המלחמה, ולזהות ערים שחלו בהן שינויים בולטים.  
                        4. **הקשר בין עצימות המלחמה לרמת הפשיעה**  
                        🚨 בחינת הקשר שבין עצימות המלחמה המבוטאת באמצעות כמות התרעות 'צבע אדום', לרמות הפשיעה בישוביי ישראל. ויזואליזציה זו מכילה את תיקי הפשיעה מפרוץ המלחמה ואילך.
                        '''
                    )

                )
            ) 
        ] 
    ) 
    return layout


In [8]:
def war_effects_layout(): 
    layout = html.Table( 
        style={"width":"100%", "height":"92vh", "border-spacing": "10px" },
        children=[ 
            html.Tr(
                # style={
                #     "text-align": "center",  # Center horizontally
                #     "vertical-align": "middle",  # Center vertically                
                # },
                children=
                [ 
                    html.Td( 
                        style={"width":"12%",  "vertical-align": "top", "text-align": "right"}, 
                        className="dash-tabs-content", 
                        children=[
                            html.Div(
                                "בחר מחוז"
                            ),
                            html.Div(
                                dcc.Dropdown(
                                    id = "war_effect_drop_down_1",
                                    options = sorted(war_effects_first_drop_down),
                                    value = None
                                )
                            ),
                            html.Div(
                                "בחר מרחב"
                            ),
                            html.Div(
                                dcc.Dropdown(
                                    id = "war_effect_drop_down_2",
                                    options =[],
                                    value = None
                                )
                            ),
                            html.Div(
                                "בחר תחנת משטרה"
                            ),
                            html.Div(
                                dcc.Dropdown(
                                    id = "war_effect_drop_down_3",
                                    options = [],
                                    value = None
                                )
                            ),
                            html.Div(
                                dcc.Checklist(
                                    id="war_effect_stat_group_values",
                                    options=[{"label": stat, "value": stat} for stat in sorted(statis_group_categories)],
                                    value= ["עבירות נגד גוף", "עבירות כלפי הרכוש", "עבירות סדר ציבורי", "עבירות תנועה", "עבירות בטחון", "עבירות מין"]
                                ),
                                style={"margin-top": "20px"}
                            ),
                            
                        ]
                    ), 
                    html.Td( 
                        style={"width":"70%", "padding": "10px"}, 
                        className="dash-tabs-content", 
                        children=[
                            dcc.Graph(
                                id='war_effect_graph',
                            )
                        ]
                    ), 
                    html.Td( 
                        style={"width":"18%",  "vertical-align": "top"}, 
                        className="dash-tabs-content",
                        children=[
                            dcc.Markdown(
                                '''
                                ### הסבר על וויזואליזצית מגמות גיאוגרפיות
                                המטרה של הוויזואליזציה היא לתת אפשרות לאנשי מקצוע או חוקרים הסתכלות על התפלגות תיקי המשטרה שנפתחו לפי קטגוריות.
                                ניתן לסנן את הנתונים לפי מחוז, מרחבים, ותחנות משטרה כדי לקבל תובנות ברורות וממוקדות.

                                ### הוראות שימוש

                                1. **בחר מחוז**: השתמשו בתיבת הבחירה בצד ימין כדי לבחור מחוז. ניתן להשאיר ריק כדי לראות את כל המחוזות.
                                2. **בחר מרחב**: אם נבחר מחוז, תיבת הבחירה של המרחבים תתעדכן אוטומטית עם האפשרויות הרלוונטיות.
                                3. **בחר תחנת משטרה**: לאחר בחירת מרחב, ניתן לבחור תחנת משטרה לקבלת מידע נוסף.
                                4. **בחר סוגי עבירות**: סמן או תבטל את סימון סוגי העבירות הרצויות כדי להתמקד בקטגוריות מסוימות.
                                5. **גרף**: הנתונים מוצגים בצורה גרפית כך שניתן לראות מגמות ותובנות.
                                '''
                            )
                        ]
                    )
                ]
            ) 
        ] 
    ) 
    return layout

In [9]:
def crime_cases_layout(): 
    layout = html.Table( 
        style={"width":"100%", "height":"92vh", "border-spacing": "10px" },
        children=[ 
            html.Tr([
                html.Td( 
                    style={"width":"82%"},
                    # className="dash-tabs-content", 
                    children=[
                        html.Table(
                            style={"width": "100%", "height": "100%", "border-spacing": "10px"},
                            children=[
                                html.Tr(
                                    html.Td(
                                        className="dash-tabs-content",
                                        children=[
                                            dcc.Graph(
                                                id='main-bar-chart',
                                                config={'displayModeBar': False}
                                            )
                                        ]
                                    )
                                ),
                                html.Tr(
                                    html.Td(
                                        className="dash-tabs-content",
                                        children=[
                                            html.Div(id='output-graph', children=dcc.Graph(id='detail-graph'))
                                        ]
                                    )
                                )
                            ]
                        )
                        # html.Div([
                        #             dcc.Graph(
                        #                 id='main-bar-chart',
                        #                 config={'displayModeBar': False}
                        #             ),
                        #             html.Div(id='output-graph', children=dcc.Graph(id='detail-graph'))
                        #         ])
                    ] 
                ),
                html.Td(
                    style={"width":"18%", "vertical-align": "top"}, 
                    className="dash-tabs-content",
                    children= [
                            dcc.Markdown(
                            '''
                            ### הסבר על וויזואליזצית תיקי פשיעה
                            מאפשר לראות הבדלים של סוגי פשיעה - שנה לפני המלחמה ושנה לתוך המלחמה.
                            ניתן לרדת לרזולוציה נמוכה יותר של סוגי הפשיעה לכל קטגוריה פשיעה כללית.
                            
                            ### הוראות שימוש
                            לחץ על עמודה על מנת להתמקד בה.
                            '''
                            )
                        ]
                )
            ]) 
        ] 
    ) 
    return layout

In [10]:
# def crime_cases_layout(): 
#     layout = html.Table( 
#         style={"width":"75%", "height":"92vh", "border-spacing": "10px", "margin": "auto"},
#         children=[ 
#             html.Tr([
#                 html.Td( 
#                     style={"width":"80%"},
#                     className="dash-tabs-content",
#                     children=[
#                         dcc.Graph(
#                             id='main-bar-chart',
#                             config={'displayModeBar': False}
#                         )
#                     ]
#                 ),
#                 html.Td(
#                     style={"width":"20%", "vertical-align": "top"}, 
#                     className="dash-tabs-content",
#                     children= [
#                         dcc.Markdown(
#                             '''
#                             ### הסבר על וויזואליזצית תיקי פשיעה
#                             מאפשר לראות הבדלים של סוגי פשיעה - שנה לפני המלחמה ושנה לתוך המלחמה.
#                             ניתן לרדת לרזולוציה נמוכה יותר של סוגי הפשיעה לכל קטגוריה פשיעה כללית.
                            
#                             ### הוראות שימוש
#                             לחץ על עמודה על מנת להתמקד בה.
#                             '''
#                         )
#                     ]
#                 )  
#             ]),
#             html.Tr(
#                 html.Td(
#                     className="dash-tabs-content",
#                     children=[
#                         html.Div(id='output-graph', children=dcc.Graph(id='detail-graph'))
#                     ]
#                 )
#             )
#         ]
#     )

#     return layout

In [11]:
def security_cases_layout(): 
    layout = html.Table( 
        style={"width":"100%", "height":"92vh", "border-spacing": "10px"},
        children=[
            # First Row (Top-left and top-middle cells merged)
            html.Tr([
                html.Td(  # Merged top-left and top-middle cells
                    style={"width":"80%", "height": "50%"},  # Combined width for left and middle
                    className="dash-tabs-content",
                    colSpan=2,  # Merge horizontally
                    children=[
                        html.Div(
                            style = {
                                "display": "flex",
                                "justifyContent": "center",  # Horizontal centering
                                "alignItems": "center",      # Vertical centering
                                "height": "100%",            # Match parent cell height
                                "width": "100%",             # Match parent cell width,
                            },
                            children=[
                                dcc.Graph(
                                    figure=make_security_cases_fig(twin_bar_data),
                                    id="security_cases_twin_bar"
                                )
                            ]
                        )
                    ]
                ),
                html.Td(  # Right column
                    style={"width":"20%", "height": "100%", "vertical-align": "top"},
                    className="dash-tabs-content",
                    rowSpan=2,  # Merge vertically
                    children=[
                        dcc.Markdown(
                        '''
                        ### הסבר על וויזואליזצית תיקי ביטחון
                        המטרה של הויזואליזציה היא לתת סקירה על השפעת המלחמה על עבירות ביטחון.
                        ניתן לצלול לתוך עבירות מסוימות ולחקור ערים בעלות שינוי מרבי או לבחור ערים לבקשתכם.
                        טווח הנתונים הוא שנה לפני המחלחמה ושנה אחרי.

                        ### הוראות שימוש

                        1. **בחר סוג עבירה**: לחץ על עמודה בגרף הראשי להצגת נתונים נוספים.
                        2. **שנו את התצוגה**: לחץ על תצוגה לוגריתמית על מנת לראות בבירור סוגי פשיעה עם מס' תיקים נמוך ולצפות בערכים המקוריים בעזרת העכבר.
                        3. **תבחר מס' ערים בסליידר**: השתמשו בסליידר על מנת לבחור מס' ערים בעלות שינוי מירבי.
                        4. **בחר ערים להשוואה**: השתמשו במחוון ותפריט הבחירה כדי למקד את ההשוואה.

                        '''
                        )

                    ]
                )
            ]),
            # Second Row (Bottom-left and bottom-middle cells separate)
            html.Tr([
                html.Td(  # Bottom-left
                    style={"width":"15%", "height": "50%"},
                    # className="dash-tabs-content",
                    children=[
                        html.Table(
                            style = {"width":"100%", "height":"100%", "border-spacing": "10px"},
                            children=[  # Wrap the `html.Tr` elements in a list
                                html.Tr(
                                    html.Td(
                                        style = {"height":"20%"},
                                        className="dash-tabs-content",
                                        children=[
                                            dcc.Checklist(
                                                id="security_cases_twin_bar_log_view",
                                                options=[{"label": "תצוגה לוגריתמית", "value": "תצוגה לוגריתמית"}],
                                                value= []
                                            )
                                        ]
                                    )
                                ),
                                html.Tr(
                                    html.Td(
                                        style = {"height":"80%", "vertical-align": "top"},
                                        className="dash-tabs-content",
                                        children= [
                                            html.Div(
                                                "טופ X ערים"
                                            ),
                                            dcc.Slider(
                                                min=0,
                                                max=10,
                                                step=1,
                                                value=5,
                                                id='security_cases_slider'
                                            ),
                                            html.Div(
                                                "ערים לבדיקה"
                                            ),
                                            dcc.Dropdown(
                                                geo_names,
                                                [],
                                                multi=True,
                                                id="security_cases_highlighted_geos"
                                            )
                                        ]
                                    )

                                )
                            ]
                        )
                    ]
                ),
                html.Td(  # Bottom-middle
                    style={"width":"68%", "height": "50%"},
                    children=[
                        html.Table(
                            style={"width":"100%", "height": "100%", "border-spacing": "10px"},
                            children=[
                                html.Tr(
                                    children=[
                                        html.Td(
                                            style={"width":"50%"},
                                            className="dash-tabs-content",
                                            children=[
                                                html.Div(
                                                    style = {
                                                        "display": "flex",
                                                        "justifyContent": "center",  # Horizontal centering
                                                        "alignItems": "center",      # Vertical centering
                                                        "height": "100%",            # Match parent cell height
                                                        "width": "100%",             # Match parent cell width,
                                                    },
                                                    id="security_cases_right_graph",
                                                    children=[

                                                    ]
                                             
                                                )

                                            ]

                                        ),
                                        html.Td(
                                            style={"width":"50%"},
                                            className="dash-tabs-content",
                                            children=[
                                                html.Div(
                                                    style = {
                                                        "display": "flex",
                                                        "justifyContent": "center",  # Horizontal centering
                                                        "alignItems": "center",      # Vertical centering
                                                        "height": "100%",            # Match parent cell height
                                                        "width": "100%",             # Match parent cell width,
                                                    },
                                                    id="security_cases_left_graph",
                                                    children=[
                                                        
                                                    ]
                                             
                                                )

                                            ]

                                        )                             
                                    ]
                                )
                            ]
                        )
                    ],
                )
            ])
        ]
    )
    return layout


In [12]:
def intensity_of_the_war_layout():
    layout = html.Table(
        style={"width": "100%", "height":"92vh", "border-spacing": "10px"},
        children=[
            html.Tr(
                [
                    html.Td(
                        # className="dash-tabs-content",
                        style={"width":"82%", "height": "100%", "vertical-align": "middle", "text-align": "center"},
                        children=[
                            html.Table(
                                style = {"width":"100%", "height": "100%", "border-spacing": "10px", "text-align": "center"},
                                children=[
                                    html.Tr(
                                        style = {"height": "10%"},
                                        children=[
                                            html.Td(
                                                style = {"width": "33%"},
                                                className="dash-tabs-content",
                                                children=[
                                                    html.Div(
                                                        "סינון ערים"
                                                    ),
                                                    html.Div(
                                                        style = {"width": "66%", "margin-right": "17%"},
                                                        children=[
                                                            dcc.Dropdown(
                                                                id='city-filter',
                                                                options=[{'label': city, 'value': city} for city in sorted(df['City'].unique())],
                                                                multi=True,
                                                                placeholder="Select cities...",
                                                                style={'width': '100%', 'height': '30px'}
                                                            )
                                                        ]
                                                    )
                                                ]
                                            ),
                                            html.Td(
                                                style = {"width": "33%"},
                                                className="dash-tabs-content",
                                                children=[
                                                    html.Div(
                                                        "טווח אזעקות"
                                                    ),
                                                    html.Div(
                                                        style = {"width": "66%", "margin-right": "17%"},
                                                        children=[
                                                            dcc.RangeSlider(
                                                                id='x-range-slider',
                                                                min=0,
                                                                max=df['x'].max() + 400,
                                                                value=[0, 100],
                                                                step=20,
                                                                marks=None,
                                                                tooltip={"placement": "bottom", "always_visible": False},
                                                                allowCross=False,
                                                            )
                                                        ]
                                                    )
                                                ]
                                            ),
                                            html.Td(
                                                style = {"width": "33%"},
                                                className="dash-tabs-content",
                                                children=[
                                                    html.Div(
                                                        "טווח תיקי פשיעה"
                                                    ),
                                                    html.Div(
                                                        style = {"width": "66%", "margin-right": "17%"},
                                                        children=[
                                                            dcc.RangeSlider(
                                                                id='y-range-slider',
                                                                min=0,
                                                                max=df['y'].max() + 400,
                                                                value=[0, 1020],
                                                                step=20,
                                                                marks=None,
                                                                tooltip={"placement": "bottom", "always_visible": False},
                                                                allowCross=False,
                                                            )
                                                        ]
                                                    )                    
                                                ]
                                            )
                                        ]
                                    ),
                                    html.Tr(
                                        style = {"height": "90%"},
                                        children=[
                                            html.Td(
                                                colSpan = 3,
                                                className="dash-tabs-content",
                                                children=[
                                                    html.H1(
                                                        "הקשר שבין מס' התרעות 'צבע אדום' לבין רמת הפשיעה ביישובים בישראל ",
                                                        style={
                                                            'textAlign': 'center',
                                                            'marginTop': '5px',
                                                            'marginBottom': '15px',
                                                            'fontSize': '20px'
                                                        }
                                                    ),
                                                    dcc.Graph(
                                                        id='quarterly-scatter-plot',
                                                        # style={'height': '500px'},
                                                        config={'displayModeBar': False}
                                                    )
                                                ]
                                            )
                                        ]
                                    )
                                ]
                            )
                        ]
                    ),
                    html.Td(
                        style={"width":"18%", "height": "100%", "vertical-align": "top" },
                        className="dash-tabs-content",
                        children = [
                            dcc.Markdown(
                                '''
                                ### הסבר על וויזואליזצית עצימות מלחמה
                                מטרת הגרף היא לראות אם יש קורלציה בין עצימות המלחמה באיזור (כמות האזעקות באיזור)
                                לבין כמות הפשיעה באיזור.
                                
                                
                                ### הוראות שימוש
                                1. בחר את הערים שתרצה להציג על הגרף
                                
                                2. ניתן בעזרת הסליידרים להסתכל על הקורלציה ברזולוציות שונות
                                
                                3. תבליט עיר בכל הרבעים על ידי השמת עכבר על נקודה 
                                
                                '''
                            )
                        ] 
                    )
                ]
            )
        ]
    )
    return layout

In [13]:
# Initialize Dash App
app = dash.Dash(__name__, suppress_callback_exceptions=True)
app.layout = html.Div(
    className="pcss3t pcss3t-theme-1",  # Apply the main CSS classes
    dir="rtl",
    children=[
        dcc.Tabs(
            id="navigation-tab",
            value="Overview",
            className="dash-tabs",
            children=[
                dcc.Tab(label="מבוא", value="Overview", className="tab", selected_className="tab--selected"),
                dcc.Tab(label="מבט על", value="Crime cases", className="tab", selected_className="tab--selected"),
                dcc.Tab(label="מגמות גיאוגרפיות", value="War effects on crime", className="tab", selected_className="tab--selected"),
                dcc.Tab(label="תיקי ביטחון", value="Security cases", className="tab", selected_className="tab--selected"),
                dcc.Tab(label="עצימות המלחמה", value="Intensity of the war", className="tab", selected_className="tab--selected")
            ]
        ),
        html.Div(
            id="tab-content",
            children="Content for the selected tab will appear here.",
            dir="rtl"
        )
    ]
)


In [14]:
# tabs callbacks
@app.callback(
    Output("tab-content", "children"),
    Input("navigation-tab", "value")
)
def update_dashboard_tab(selected_tab):
    if selected_tab == "Overview":
        layout = overview_layout()
    elif selected_tab == "War effects on crime":
        layout = war_effects_layout()
    elif selected_tab == "Crime cases":
        layout = crime_cases_layout()
    elif selected_tab == "Security cases":
        layout = security_cases_layout()
    elif selected_tab == "Intensity of the war":
        layout = intensity_of_the_war_layout()        
    else:
        layout = html.Div(children=[
        html.Div(
            children="No content available.",  # Default content for unmatched tabs
            className="dash-tabs-content"
        ),
        html.Div(
            children="No content available.",  # Default content for unmatched tabs
            className="dash-tabs-content"
        )
        ])
    return layout


In [15]:
# Dashboard 1 Callbacks
def make_war_effect_fig(df, filter_value):
    df = df.groupby([filter_value, 'Year', 'Quarter', 'StatisticGroup']).size().reset_index(name='CrimeAmount')

    df['YearQuarter'] = df['Quarter'].astype(str) + '/' + df['Year'].astype(str).str[-2:]
    df['hover_text'] = (
    'סוג פשע: ' + df['StatisticGroup'] +
    '<br>' + df['YearQuarter'] + ': רבעון ושנה' +
    "<br>" + df['CrimeAmount'].astype(str) + ": מס' תיקי פשיעה"
    )  

    full_quarters = [f"Q{q}/{str(year)[-2:]}" for year in range(2022, 2024 + 1) for q in range(1, 5)]
    subplots_num = len(df[filter_value].unique())

    fig = px.line(
        df,
        x='YearQuarter',
        y='CrimeAmount',
        color='StatisticGroup',
        facet_col=filter_value,
        facet_col_wrap=3,
        labels={'YearQuarter': 'Year-Quarter', 'CrimeAmount': "מס' תיקי פשיעה"},
        facet_row_spacing=0.1,
        facet_col_spacing=0.05,
        # hover_data={'hover_text': True},  # Use the custom hover_text column
        custom_data=["StatisticGroup"],
        height= 500 if subplots_num <= 3 else 700 if subplots_num <= 6 else 800
    )
    fig.update_xaxes(showticklabels=True, tickfont=dict(size=10), title=None, autorange="reversed", showgrid=False)
    fig.update_yaxes(showticklabels=True, tickfont=dict(size=10), side="right", title=None, showgrid=False, range=[0, None])
    for annotation in fig.layout.annotations:
        annotation.text = annotation.text.split('=')[1]  # Keep only the value
        annotation.font = dict(size=16, color='black', family='Arial', weight='bold')  # Make bold and adjust font

    fig.update_layout(
        legend=dict(
            orientation="h",  # Horizontal legend
            x=0.5,  # Center the legend horizontally
            y=-0.2,  # Position the legend below the plot
            xanchor="center",  # Anchor the x position to the center
            yanchor="top"  # Anchor the y position to the top
        ),
        legend_title=dict(
            text=" סוגי פשיעה ",  # The new legend title text
            font=dict(size=18, family="Arial", weight="bold"),  # Customize the font of the title
            side="top right"
        ),

        plot_bgcolor="#f9f9f9",  # Set the background of the plot to white
        paper_bgcolor="white",  # Set the background of the entire figure to white
        xaxis=dict(
            type='category',
            categoryorder='array',
            categoryarray=full_quarters,
        ),
    )
    fig.for_each_xaxis(lambda axis: axis.update(showline=True, linecolor="black"))
    fig.for_each_yaxis(lambda axis: axis.update(showline=True, linecolor="black"))

    fig.update_traces(mode='lines+markers', hovertemplate="סוג פשע: %{customdata[0]}<br>מס' תיקים: %{y}<extra></extra>")#, hovertemplate='%{customdata}<extra></extra>', customdata=df['hover_text'])  # Adds markers to the lines

    fig.add_annotation(
        x=1.07,  # Align to the left
        y=0.5,  # Center vertically
        text="מס' תיקי פשיעה",  # The Y-axis title
        showarrow=False,
        textangle=-270,  # Rotate vertically
        xref="paper",  # Relative to the figure
        yref="paper",
        font=dict(size=24, family="Arial", weight="bold")
    )
    return fig

@app.callback(
        [
            Output("war_effect_graph", "figure"),
            Output("war_effect_drop_down_2", "options"),
            Output("war_effect_drop_down_2", "value"),
            Output("war_effect_drop_down_3", "options"),
            Output("war_effect_drop_down_3", "value")
        ],
        [
            Input("war_effect_stat_group_values", "value"),
            Input("war_effect_drop_down_1", "value"),
            Input("war_effect_drop_down_2", "value"),
            Input("war_effect_drop_down_3", "value")
        ]
)
def update_war_effects_dashboard(stat_group_selected, war_effect_drop_down_1_value, war_effect_drop_down_2_value, war_effect_drop_down_3_value):
    temp_df = war_effects_df[war_effects_df['StatisticGroup'].isin(stat_group_selected)]

    if not war_effect_drop_down_1_value:
        fig = make_war_effect_fig(temp_df, 'PoliceDistrict')
        return fig, [], None, [], None
    
    # If first not None
    drop_down_2_options = merged_df[merged_df['PoliceDistrict'] == war_effect_drop_down_1_value]['PoliceMerhav'].unique()
    temp_df = temp_df[temp_df["PoliceDistrict"] == war_effect_drop_down_1_value]

    if not war_effect_drop_down_2_value:
        fig = make_war_effect_fig(temp_df, 'PoliceMerhav')
        return fig, sorted(drop_down_2_options), None, [], None
    
    # If second not None
    drop_down_3_options = merged_df[(merged_df['PoliceDistrict'] == war_effect_drop_down_1_value) & (merged_df['PoliceMerhav'] == war_effect_drop_down_2_value)]['PoliceStation'].unique()
    temp_df = temp_df[temp_df["PoliceMerhav"] == war_effect_drop_down_2_value]

    if not war_effect_drop_down_3_value:
        fig = make_war_effect_fig(temp_df, 'PoliceStation')
        return fig, sorted(drop_down_2_options), war_effect_drop_down_2_value, sorted(drop_down_3_options), None
    
    #If all are not None
    temp_df = temp_df[temp_df["PoliceStation"] == war_effect_drop_down_3_value]
    fig = make_war_effect_fig(temp_df, 'GeoName')
    return fig , sorted(drop_down_2_options), war_effect_drop_down_2_value, sorted(drop_down_3_options), war_effect_drop_down_3_value

In [16]:
# dashboard 2 callbacks

@app.callback(
    Output('main-bar-chart', 'figure'),
    Input('main-bar-chart', 'clickData')  # Now triggered by click data
)
def create_main_bar_chart(clickData):
    # Determine selected group from click data
    selected_group = clickData['points'][0]['x'] if clickData else None
    
    # Create opacity arrays (1.0 for selected, 0.3 for others)
    opacity_pre = [1.0 if group == selected_group else 0.3 for group in pre_war_counts['StatisticGroup']] if selected_group else [1.0]*len(pre_war_counts)
    opacity_post = [1.0 if group == selected_group else 0.3 for group in post_war_counts['StatisticGroup']] if selected_group else [1.0]*len(post_war_counts)

    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=pre_war_counts['StatisticGroup'],
        y=pre_war_counts['Count'],
        name='לפני המלחמה',
        marker_color='blue',
        marker_opacity=opacity_pre  # Apply opacity array
    ))
    fig.add_trace(go.Bar(
        x=post_war_counts['StatisticGroup'],
        y=post_war_counts['Count'],
        name='אחרי המלחמה',
        marker_color='red',
        marker_opacity=opacity_post  # Apply opacity array
    ))
    fig.update_layout(
        barmode='group',
        xaxis_title='סוגי עבירות',
        yaxis_title=None,
        title={
            'text': f"כמות תיקי הפשיעה לפי סוגי עבירות",
            'x': 0.5,                  # Center horizontally (0 = left, 1 = right)
            'xanchor': 'center',       # Anchor the title's position
            'y': 0.85,                 # Adjust the vertical position (optional, default is near top)
            'yanchor': 'top'           # Anchor vertically relative to y position
        },
        height = 400,
        legend=dict(
            x=1,  # Center the legend horizontally
            y=1.2,
            xanchor="center",  # Anchor the x position to the center
            yanchor="middle"  # Anchor the y position to the top
        ),
        title_font = dict(weight="bold")
        
    )
    fig.update_yaxes(showticklabels=True, tickfont=dict(size=10), side="right")
    fig.add_annotation(
        x=1.08,  # Align to the left
        y=0.5,  # Center vertically
        text= "מס' תיקי פשיעה",
        showarrow=False,
        textangle=-270,  # Rotate vertically
        xref="paper",  # Relative to the figure
        yref="paper",
        font=dict(size=16, family="Arial")
    )
    fig.update_traces(hovertemplate = "מס' תיקי פשיעה: %{y:.0f}<br>סוג עבירה: %{x}")
    return fig

# Callback to update the detailed graph based on clicked category
@app.callback(
    Output('detail-graph', 'figure'),
    Input('main-bar-chart', 'clickData')
)
def update_detail_graph(clickData):
    if clickData is None:
        # Default figure if no category is clicked
        return go.Figure().update_layout(
                title={
                    'text': "לחץ על קטגוריית סוג פשע על מנת לראות פרטים עליה",
                    'x': 0.5,                  # Center horizontally (0 = left, 1 = right)
                    'xanchor': 'center',       # Anchor the title's position
                    'y': 0.85,                 # Adjust the vertical position (optional, default is near top)
                    'yanchor': 'top'           # Anchor vertically relative to y position
                },
                height=400
            )

    # Extract the clicked category (StatisticGroup)
    statistic_group = clickData['points'][0]['x']

    # Filter data for the clicked category
    pre_war_df_specific = pre_war_df[pre_war_df['StatisticGroup'] == statistic_group]
    post_war_df_specific = post_war_df[post_war_df['StatisticGroup'] == statistic_group]

    # Group by StatisticType and count rows
    pre_war_details = pre_war_df_specific.groupby('StatisticType').size().reset_index(name='Count')
    post_war_details = post_war_df_specific.groupby('StatisticType').size().reset_index(name='Count')

    pre_war_details = pre_war_details.sort_values(by='StatisticType', ascending=True)
    post_war_df_specific = post_war_df_specific.sort_values(by='StatisticType', ascending=True)

    # Create a detailed graph for the clicked category
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=pre_war_details['StatisticType'],
        y=pre_war_details['Count'],
        name='לפני המלחמה',
        marker_color='blue'
    ))
    fig.add_trace(go.Bar(
        x=post_war_details['StatisticType'],
        y=post_war_details['Count'],
        name='אחרי המלחמה',
        marker_color='red'
    ))
    fig.update_layout(
        barmode='group',
        xaxis_title=f'סוגי {statistic_group}',
        yaxis_title= None,
        title={
            'text': f"כמות תיקי פשיעה בקטגוריית {statistic_group}",
            'x': 0.5,                  # Center horizontally (0 = left, 1 = right)
            'xanchor': 'center',       # Anchor the title's position
            'y': 0.85,                 # Adjust the vertical position (optional, default is near top)
            'yanchor': 'top'           # Anchor vertically relative to y position
            
        },
        height = 400,
        legend=dict(
            x=1,  # Center the legend horizontally
            y=1.2,
            xanchor="center",  # Anchor the x position to the center
            yanchor="middle"  # Anchor the y position to the top
        ),
        title_font = dict(weight="bold")
    )
    fig.update_yaxes(showticklabels=True, tickfont=dict(size=10), side="right")
    fig.add_annotation(
        x=1.08,  # Align to the left
        y=0.5,  # Center vertically
        text= "מס' תיקי פשיעה",
        showarrow=False,
        textangle=-270,  # Rotate vertically
        xref="paper",  # Relative to the figure
        yref="paper",
        font=dict(size=16, family="Arial")
    )
    fig.update_traces(hovertemplate = "מס' תיקי פשיעה: %{y:.0f}<br>סוג " + statistic_group + ": %{x}")

    return fig


In [17]:
# dashboard 3 callbacks
def make_diff_graph(df, statistic_type, geo_names, diff_mode="absolute", Ngeo=5):
    """
    Creates a bar graph showing differences for GeoNames, filtered by a specific StatisticType.
    
    Parameters:
    - df: The DataFrame containing the data.
    - statistic_type: The StatisticType to filter on.
    - geo_names: List of GeoNames (cities) to include in addition to the top 5.
    - diff_mode: Mode of difference calculation ("absolute" for delta, "relative" for percentage).
    
    Returns:
    - fig: A Plotly figure object.
    """


    # Step 1: Filter by the selected StatisticType
    filtered_df = df[df['StatisticType'] == statistic_type]

    # Step 2: Group by GeoName and TimePeriod to calculate total CrimeAmount
    grouped_df = filtered_df.groupby(['GeoName', 'TimePeriod']).size().reset_index(name='CrimeAmount')

    after_values = grouped_df[grouped_df["TimePeriod"] == "אחרי"]
    before_values = grouped_df[grouped_df["TimePeriod"] == "לפני"]

    # Merge the before and after values on GeoName
    merged_values = pd.merge(
        before_values[['GeoName', 'CrimeAmount']],
        after_values[['GeoName', 'CrimeAmount']],
        on='GeoName',
        how='outer',
        suffixes=('_Before', '_After')
    )
    # Fill NaN values with 0 for cities that exist only in one period
    merged_values.fillna(0, inplace=True)

    # Step 4: Calculate differences
    if diff_mode == "absolute":
        merged_values['Difference'] = merged_values['CrimeAmount_After'] - merged_values['CrimeAmount_Before']
    elif diff_mode == "relative":
        merged_values['Difference'] = ((merged_values['CrimeAmount_After']) / merged_values['CrimeAmount_Before'].replace(0, float('inf'))) * 100

    # Rename columns for clarity
    merged_values.rename(
        columns={
            'CrimeAmount_Before': 'BeforeCrimeAmount',
            'CrimeAmount_After': 'AfterCrimeAmount'
        },
        inplace=True
    )

    # Step 5: Sort by Difference and identify the top 5 GeoNames
    top_5_geo = merged_values.nlargest(Ngeo, 'Difference')

    # Step 6: Include additional GeoNames specified by the user
    filtered_geo = pd.concat([top_5_geo, merged_values[merged_values['GeoName'].isin(geo_names)]]).drop_duplicates()

    # Step 8: Sort filtered_geo by Difference in ascending order
    filtered_geo = filtered_geo.sort_values(by='Difference', ascending=True)

    # Step 9: Create the bar chart with custom hovertemplate
    fig = px.bar(
        filtered_geo,
        x='GeoName',
        y='Difference',
        color='GeoName',  # Use color to distinguish GeoNames
        labels={'Difference': 'Difference', 'GeoName': 'מיקום גיאוגרפי'},
        title= "הבדל " + ("אבסולוטי" if diff_mode == "absolute" else "באחוזים") + "<br>לפי מקום גיאוגרפי " + f"{statistic_type}",
        width=500,
        height=400,
        custom_data=['BeforeCrimeAmount', 'AfterCrimeAmount']
    )

    fig.update_yaxes(showticklabels=True, tickfont=dict(size=10), side="right", title=None, ticksuffix=None if diff_mode == "absolute" else "%")


    if diff_mode == "absolute":
        fig.update_traces(hovertemplate = "הבדל: %{y}<br>לפני: %{customdata[0]}<br>אחרי: %{customdata[1]}<extra></extra>")
    else:
        fig.update_traces(hovertemplate = "הבדל: %{y}<br>לפני: %{customdata[0]}<br>אחרי: %{customdata[1]}<extra></extra>")


    fig.update_layout(
        showlegend=False,
        title={
            'x': 0.5,                  # Center horizontally (0 = left, 1 = right)
            'xanchor': 'center',       # Anchor the title's position
            'y': 0.85,                 # Adjust the vertical position (optional, default is near top)
            'yanchor': 'top'           # Anchor vertically relative to y position
        },
        height=400,
        width =400
    )

    fig.add_annotation(
        x=1.25,  # Align to the left
        y=0.5,  # Center vertically
        text= ("כמות" if diff_mode == "absolute" else "אחוז") + " עלייה בתיקים מאז המלחמה",  # The Y-axis title
        showarrow=False,
        textangle=-270,  # Rotate vertically
        xref="paper",  # Relative to the figure
        yref="paper",
        font=dict(size=16, family="Arial")
    )
    return fig



@app.callback(
    Output("security_cases_twin_bar", "figure"),
    [
        Input("security_cases_twin_bar_log_view", "value"),
        Input("security_cases_twin_bar", "clickData"),
    ]
)
def update_secury_cases_twin_bar(is_log, stat_type):
    if stat_type:
        statisticType = stat_type["points"][0]["x"]
    else:
        statisticType = None
    if is_log:
        return make_security_cases_fig(twin_bar_data, True, highlight_statistic = statisticType)
    return make_security_cases_fig(twin_bar_data, highlight_statistic = statisticType)


@app.callback(
    [
        Output("security_cases_left_graph", "children"),
        Output("security_cases_right_graph", "children")
    ],
    [
        Input("security_cases_twin_bar", "clickData"),
        Input("security_cases_highlighted_geos", "value"),
        Input("security_cases_slider", "value"),
    ]
)
def update_secury_cases(statisticTypeClicked, highlighted_geos, topNgeo):
    if not statisticTypeClicked:
        return None, None
    statisticType = statisticTypeClicked["points"][0]["x"]
    left_figure = make_diff_graph(bitahon_df, statisticType, highlighted_geos, diff_mode="absolute", Ngeo=topNgeo)
    right_figure = make_diff_graph(bitahon_df, statisticType, highlighted_geos, diff_mode="relative", Ngeo=topNgeo)


    return dcc.Graph(figure=left_figure, config={"displayModeBar": False}), dcc.Graph(figure=right_figure, config={"displayModeBar": False})

In [18]:
# dashboard 4 callbacks

@app.callback(
    Output('quarterly-scatter-plot', 'figure'),
    [Input('quarterly-scatter-plot', 'hoverData'),
     Input('city-filter', 'value'),
     Input('x-range-slider', 'value'),
     Input('y-range-slider', 'value')]
)
def update_figure(hover_data, selected_cities, x_range, y_range):
    hovered_city = None
    tooltip_text = ""

    # Handle hover data
    if hover_data and 'points' in hover_data and hover_data['points']:
        point = hover_data['points'][0]
        hovered_city = point.get('customdata', [None])[0]

        if hovered_city:
            city_data = df[df['City'] == hovered_city]
            tooltip_lines = [f"<b>{hovered_city}</b>"]
            for quarter in ['Q4', 'Q1', 'Q2', 'Q3']:
                q_data = city_data[city_data['Quarter'] == quarter]
                if not q_data.empty:
                    tooltip_lines.append(
                        f"{quarter}: Red: {q_data['x'].values[0]} | Crime: {q_data['y'].values[0]}"
                    )
            tooltip_text = "<br>".join(tooltip_lines)

    # Filter data based on selected cities
    filtered_df = df[df['City'].isin(selected_cities)] if selected_cities else df

    # Get range values from sliders
    x_min, x_max = x_range
    y_min, y_max = y_range

    # Create subplots
    fig = make_subplots(
        rows=2,
        cols=2,
        subplot_titles=['Q4/2023', 'Q1/2024', 'Q2/2024', 'Q3/2024'],
        vertical_spacing=0.25,
        horizontal_spacing=0.2,
        shared_xaxes=False,
        shared_yaxes=False
    )

    # Plot each quarter
    quarters = ['Q4', 'Q1', 'Q2', 'Q3']
    for i, quarter in enumerate(quarters):
        row = (i // 2) + 1
        col = (i % 2) + 1
        quarter_df = filtered_df[filtered_df['Quarter'] == quarter]

        # Adjust marker appearance based on hover
        sizes = [14 if city == hovered_city else 8 for city in quarter_df['City']]
        opacities = [1.0 if city == hovered_city else 0.7 for city in quarter_df['City']]

        fig.add_trace(go.Scatter(
            x=quarter_df['x'],
            y=quarter_df['y'],
            mode='markers',
            marker={
                'size': sizes,
                'color': ['red' if city == hovered_city else 'black' for city in quarter_df['City']],
                'opacity': opacities,
                'line': {'width': 1, 'color': 'white'}
            },
            customdata=quarter_df[['City']],
            hoverinfo='text',
            hovertext=[tooltip_text if city == hovered_city else city for city in quarter_df['City']]
        ), row=row, col=col)

    # Update layout
    fig.update_layout(
        # height=600,
        # width=1400,
        showlegend=False,
        plot_bgcolor='white',
        margin=dict(l=50, r=30, t=60, b=50),
        hovermode='closest',
        hoverlabel=dict(
            bgcolor="white",
            font_size=12,
            font_family="Arial",
            bordercolor='black',
            align='left',
            namelength=-1
        )
    )

    # Update axes for all subplots
    for row in 1, 2:
        for col in 1, 2:
            fig.update_xaxes(
                row=row, col=col,
                # range=[x_min, x_max],
                range=[x_max, x_min],

                title="אזעקות",
                showgrid=True,
                gridcolor='#f0f0f0',
                linecolor='black',
                title_standoff=15,
                # autorange="reversed"
            )
            fig.update_yaxes(
                row=row, col=col,
                range=[y_min, y_max],
                title=None,
                showgrid=True,
                gridcolor='#f0f0f0',
                linecolor='black',
                title_standoff=15,
                side="right"
            ),
            fig.add_annotation(
                # y=0.8,  # Center vertically
                text="מס' תיקי פשיעה",  # The Y-axis title
                showarrow=False,
                textangle=-270,  # Rotate vertically
                xref=f"x{(row - 1) * 2 + col}",  # Reference specific subplot x-axis
                yref=f"y{(row - 1) * 2 + col}",  # Reference specific subplot y-axis
                font=dict(size=16, family="Arial"),
                x=1.8,  # Align to the left
                xshift=50


            )

    return fig

In [19]:
# Run the app
if __name__ == '__main__':
    # # Set your ngrok authtoken
    # conf.get_default().auth_token = "2rXRkgXJ1mMiT05zKxWrXMRywQB_7QpEPM58LVzseYYzA5qcJ"
    # # Start ngrok tunnel on port 8050 (default Dash port)
    
    # public_url = ngrok.connect(8050)
    # print(f"Public ngrok URL: {public_url}")
    app.run_server(debug=False)

Public ngrok URL: NgrokTunnel: "https://a814-147-235-205-25.ngrok-free.app" -> "http://localhost:8050"


t=2025-01-29T21:28:07+0200 lvl=warn msg="failed to check for update" obj=updater err="Post \"https://update.equinox.io/check\": context deadline exceeded"
